In [7]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pyswarms as ps
import os

# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load dataset
features = pd.read_csv("weatherAUS_cleaned.csv")
X = features.drop(["RainTomorrow"], axis=1).values
y = features["RainTomorrow"].values

# Standardize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1).to(device)

# Define PyTorch ANN model
class ANNModel(nn.Module):
    def __init__(self, input_size, neurons_layer1, neurons_layer2, neurons_layer3, dropout_rate1, dropout_rate2):
        super(ANNModel, self).__init__()
        self.fc1 = nn.Linear(input_size, int(neurons_layer1))
        self.fc2 = nn.Linear(int(neurons_layer1), int(neurons_layer2))
        self.fc3 = nn.Linear(int(neurons_layer2), int(neurons_layer3))
        self.dropout1 = nn.Dropout(dropout_rate1)
        self.fc4 = nn.Linear(int(neurons_layer3), 8)
        self.dropout2 = nn.Dropout(dropout_rate2)
        self.fc5 = nn.Linear(8, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.dropout1(x)
        x = torch.relu(self.fc4(x))
        x = self.dropout2(x)
        x = self.sigmoid(self.fc5(x))
        return x

# Objective function for PSO
def evaluate(params):
    neurons_layer1, neurons_layer2, neurons_layer3, dropout_rate1, dropout_rate2 = params.T
    
    neurons_layer1 = np.clip(neurons_layer1, 16, 128).astype(int)
    neurons_layer2 = np.clip(neurons_layer2, 16, 128).astype(int)
    neurons_layer3 = np.clip(neurons_layer3, 8, 64).astype(int)
    dropout_rate1 = np.clip(dropout_rate1, 0.1, 0.5)
    dropout_rate2 = np.clip(dropout_rate2, 0.1, 0.5)
    
    model = ANNModel(X_train.shape[1], neurons_layer1[0], neurons_layer2[0], neurons_layer3[0], dropout_rate1[0], dropout_rate2[0]).to(device)
    criterion = nn.BCELoss()
    optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)
    
    model.train()
    for epoch in range(50):  # Reduced for efficiency
        optimizer.zero_grad()
        outputs = model(X_train_tensor)
        loss = criterion(outputs, y_train_tensor)
        loss.backward()
        optimizer.step()
    
    model.eval()
    with torch.no_grad():
        predictions = model(X_test_tensor)
        predictions = (predictions >= 0.5).float()
        accuracy = (predictions == y_test_tensor).float().mean().item()
    
    return -accuracy  # Minimize the negative accuracy

# Set PSO parameters
options = {'c1': 1.5, 'c2': 1.5, 'w': 0.7}

# Define the bounds for hyperparameters
bounds = (np.array([16, 16, 8, 0.1, 0.1]), np.array([128, 128, 64, 0.5, 0.5]))

# Initialize and run PSO
optimizer = ps.single.GlobalBestPSO(n_particles=10, dimensions=5, options=options, bounds=bounds)
best_cost, best_params = optimizer.optimize(evaluate, iters=20)

# Extract best hyperparameters
best_neurons_layer1, best_neurons_layer2, best_neurons_layer3, best_dropout_rate1, best_dropout_rate2 = best_params

print(f"Best Hyperparameters:")
print(f"Neurons Layer 1: {int(best_neurons_layer1)}")
print(f"Neurons Layer 2: {int(best_neurons_layer2)}")
print(f"Neurons Layer 3: {int(best_neurons_layer3)}")
print(f"Dropout Rate 1: {best_dropout_rate1:.2f}")
print(f"Dropout Rate 2: {best_dropout_rate2:.2f}")


Using device: cuda


2025-02-14 22:38:19,275 - pyswarms.single.global_best - INFO - Optimize for 20 iters with {'c1': 1.5, 'c2': 1.5, 'w': 0.7}
pyswarms.single.global_best: 100%|█████████████████████████████████████████████████████████████|20/20, best_cost=-0.815
2025-02-14 22:38:29,030 - pyswarms.single.global_best - INFO - Optimization finished | best cost: -0.8150776028633118, best pos: [99.44376633 44.21716791 45.77967535  0.48916352  0.42055684]


Best Hyperparameters:
Neurons Layer 1: 99
Neurons Layer 2: 44
Neurons Layer 3: 45
Dropout Rate 1: 0.49
Dropout Rate 2: 0.42
